In [10]:
import pandas as pd
import sqlite3

In [11]:
# Using a traditional SQL database setup with the 5 input features as our columns. We sourced this data from the Nasdaq and Yahoo Finance.

In [12]:
import pandas as pd
import sqlite3

# Read the CSV file into a DataFrame
df = pd.read_csv('sp500.csv')
df.columns = ['date', 'open', 'high', 'low', 'close']

conn = sqlite3.connect('sp500_data.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS sp500_data;")

create_table = """
CREATE TABLE IF NOT EXISTS sp500_data (
    date TEXT,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT
);
"""
cursor.execute(create_table)

insert_query = """
INSERT INTO sp500_data (date, open, high, low, close)
VALUES (?, ?, ?, ?, ?);
"""
cursor.executemany(insert_query, df.values.tolist())

conn.commit()

cursor.execute("SELECT * FROM sp500_data LIMIT 10;")
for row in cursor.fetchall():
    print(row)

conn.close()


('12/05/2024', 6075.11, 6089.03, 6094.55, 6072.9)
('12/04/2024', 6086.49, 6069.39, 6089.84, 6061.06)
('12/03/2024', 6049.88, 6042.97, 6052.07, 6033.39)
('12/02/2024', 6047.15, 6040.11, 6053.58, 6035.33)
('11/29/2024', 6032.38, 6003.98, 6044.17, 6003.98)
('11/27/2024', 5998.74, 6014.11, 6020.16, 5984.87)
('11/26/2024', 6021.63, 6000.03, 6025.42, 5992.27)
('11/25/2024', 5987.37, 5992.28, 6020.75, 5963.91)
('11/22/2024', 5969.34, 5944.36, 5972.9, 5944.36)
('11/21/2024', 5948.71, 5940.58, 5963.32, 5887.26)


In [13]:
# We use an Alpha Vantage's API to collect real time data on the S&P 500 stock for a mixture of historical data spanning the past 5 years and live data.

In [14]:
import requests
import pandas as pd

API_KEY = 'LOMS9J3D6DY37GDU'  
BASE_URL = 'https://www.alphavantage.co/query'

def get_sp500_data(interval='1min', output_size='compact'):
    """
    Fetches S&P 500 (SPY) time series data from Alpha Vantage API.

    Parameters:
    - interval (str): Time interval (e.g., '1min', '5min', '15min', '30min', '60min', 'daily', 'weekly', 'monthly')
    - output_size (str): Data size ('compact' for latest 100 data points, 'full' for entire history)

    Returns:
    - pandas.DataFrame: S&P 500 data as a DataFrame
    """
    # Defining the parameters for the API request
    params = {
        'function': 'TIME_SERIES_INTRADAY' if interval in ['1min', '5min', '15min', '30min', '60min'] else 'TIME_SERIES_DAILY',
        'symbol': 'SPY',  # SPY represents the S&P 500 ETF
        'interval': interval if interval in ['1min', '5min', '15min', '30min', '60min'] else None,
        'apikey': API_KEY,
        'outputsize': output_size
    }

    params = {k: v for k, v in params.items() if v is not None}

    response = requests.get(BASE_URL, params=params)

    response.raise_for_status()

    data = response.json()

    time_series_key = f"Time Series ({interval})" if 'interval' in params else "Time Series (Daily)"
    if time_series_key not in data:
        raise ValueError("Invalid response from Alpha Vantage API. Check your API key and parameters.")

    df = pd.DataFrame.from_dict(data[time_series_key], orient='index')

    df.columns = ['open', 'high', 'low', 'close', 'volume']

    df.index = pd.to_datetime(df.index)

    df = df.astype(float)

    return df

try:
    sp500_data = get_sp500_data(interval='daily', output_size='compact')  
    print(sp500_data.head())  
except Exception as e:
    print(f"An error occurred: {e}")


              open    high      low   close      volume
2024-12-06  607.44  609.07  607.020  607.81  31241549.0
2024-12-05  607.66  608.48  606.305  606.66  28762183.0
2024-12-04  605.63  607.91  604.950  607.66  42787561.0
2024-12-03  603.39  604.16  602.341  603.91  26906629.0
2024-12-02  602.97  604.32  602.470  603.63  31745989.0


In [15]:
# These are the features we will use to tune our Random Forest Regression model

In [16]:
def create_features(df):
    df["return"] = df["close"].pct_change()
    df["volatility"] = abs(df["high"] - df["low"])
    df['ma_5'] = df['close'].rolling(window=5).mean() 
    df['ma_10'] = df['close'].rolling(window=10).mean()  
    df = df.dropna()  
    return df.head(10)
create_features(df)
    

,date,open,high,low,close,return,volatility,ma_5,ma_10
9,11/21/2024,5948.71,5940.58,5963.32,5887.26,-0.009606,22.74,5954.534,5997.933
10,11/20/2024,5917.11,5914.34,5920.67,5860.56,-0.004535,6.33,5929.672,5976.699
11,11/19/2024,5916.98,5870.05,5923.51,5855.29,-0.000899,53.46,5902.276,5956.122
12,11/18/2024,5893.62,5874.17,5908.12,5865.95,0.001821,33.95,5882.684,5939.378
13,11/15/2024,5870.62,5912.79,5915.32,5853.01,-0.002206,2.53,5864.414,5921.146
14,11/14/2024,5949.17,5989.68,5993.88,5942.28,0.015252,4.20,5875.418,5914.976
15,11/13/2024,5985.38,5985.75,6008.19,5965.91,0.003977,22.44,5896.488,5913.080
16,11/12/2024,5983.99,6003.60,6009.92,5960.08,-0.000977,6.32,5917.446,5909.861
17,11/11/2024,6001.35,6008.86,6017.31,5986.69,0.004465,8.45,5941.594,5912.139
18,11/08/2024,5995.54,5976.76,6012.45,5976.76,-0.001659,35.69,5966.344,5915.379


In [17]:
#Trains a Random Forest regression model to predict the 'close' price of a stock or financial instrument based on several input features. We chose Random Forest regression as our model of choice because the changes between data is not linear and this model best fits the data.

In [ ]:
import pandas as pd
import sqlite3
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

def train_model(df):
    # Features and target
    X = df[['open', 'high', 'low', 'return', 'ma_5', 'ma_10']]
    y = df['close']

    # Training data vs testing data split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    feature_importance = model.feature_importances_
    features = X.columns
    plt.figure(figsize=(8, 6))
    sns.barplot(x=feature_importance, y=features, palette='viridis')
    plt.title('Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Features')
    plt.show()
    
        # Plotting Actual vs Predicted values
    plt.figure(figsize=(8, 6))
    plt.plot(y_test.values, label='Actual', color='blue')
    plt.plot(y_pred, label='Predicted', color='red', linestyle='--')
    plt.title('Actual vs Predicted Closing Prices')
    plt.xlabel('Samples')
    plt.ylabel('Close Price')
    plt.legend()
    plt.show()
    
    residuals = y_test - y_pred
    plt.figure(figsize=(8, 6))
    sns.histplot(residuals, bins=30, kde=True, color='green')
    plt.title('Distribution of Residuals (Prediction Errors)')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.show()


    return model

train_model(df)